In [9]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ---------- Configuración ----------
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

# Si ya iniciaste sesión antes en Google en tu Chrome local (recomendado para evitar CAPTCHA):
# options.add_argument(r"--user-data-dir=C:\Users\TU_USUARIO\AppData\Local\Google\Chrome\User Data")
# options.add_argument("--profile-directory=Default")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

# ---------- Datos ----------
url = 'https://pucp.kissflow.com/view/dataset/Base_de_datos_panetones_2'
url2= 'https://gmail.com'
df = pd.read_csv("Kissflow.csv", dtype=str)
usuario = df.loc[0, "usuario"]
contrasena = df.loc[0, "contrasena"]

try:
    # 1️⃣ Abrir la página de Kissflow
    driver.get(url)

    # 2️⃣ Hacer clic en "Continuar con Google"
    google_btn = wait.until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(@class,'socialLoginItem')]//span[normalize-space()='Google']"))
    )
    google_btn.click()

    # 3️⃣ Esperar a que cargue la página de login de Google y escribir el correo
    email_input = wait.until(EC.element_to_be_clickable((By.ID, "identifierId")))
    email_input.click()
    email_input.clear()
    email_input.send_keys(usuario)

    siguiente_btn = wait.until(EC.element_to_be_clickable((By.ID, "identifierNext")))
    siguiente_btn.click()

    # 4️⃣ Detectar posible CAPTCHA
    time.sleep(2)
    recaptcha_iframes = driver.find_elements(By.XPATH, "//iframe[contains(@src,'recaptcha') or contains(@src,'/challenge')]")
    challenge_elements = driver.find_elements(By.XPATH, "//*[contains(text(),'Escribe el texto') or contains(@aria-label,'captcha')]")

    if recaptcha_iframes or challenge_elements:
        print("⚠️ Se detectó un CAPTCHA. Resuélvelo manualmente y presiona ENTER para continuar.")
        input("Presiona ENTER cuando termines el CAPTCHA...")
    else:
        # 5️⃣ Si no hay CAPTCHA, esperar la contraseña y continuar
        password_input = wait.until(EC.visibility_of_element_located((By.NAME, "password")))
        password_input.click()
        password_input.clear()
        password_input.send_keys(contrasena)

        password_next = wait.until(EC.element_to_be_clickable((By.ID, "passwordNext")))
        password_next.click()

    print("✅ Login a Kissflow completado (verifica visualmente que haya ingresado).")

except Exception as e:
    print("❌ Error durante el flujo:", e)

finally:
    # No cerramos para inspeccionar el navegador
    # driver.quit()
    pass


⚠️ Se detectó un CAPTCHA. Resuélvelo manualmente y presiona ENTER para continuar.
✅ Login a Kissflow completado (verifica visualmente que haya ingresado).


In [ ]:
# Buscar todos los botones con texto "Exportar"
exportar_botones = driver.find_elements(By.XPATH, "//button[.//span[normalize-space()='Exportar']]")

# Hacer clic en el primero (por ejemplo, el que abre el modal)
exportar_botones[0].click()

# Pequeña pausa si el segundo aparece después del primero
time.sleep(1)

exportar_btn = driver.find_element(By.XPATH, "//button[contains(@class,'primary') and .//span[normalize-space()='Exportar']]")
exportar_btn.click()







In [3]:
# Abrir Gmail en nueva pestaña
driver.execute_script("window.open('https://mail.google.com', '_blank');")
driver.switch_to.window(driver.window_handles[-1])

# Esperar que cargue la bandeja
time.sleep(5)

# (Opcional) Hacer clic en el primer correo de la lista
primero = driver.find_element(By.CSS_SELECTOR, "tr.zA")  # Cada correo en Gmail tiene clase 'zA'
primero.click()



In [5]:
# localizar el enlace por parte del href (usa un fragmento estable del URL)
link = driver.find_element(By.XPATH, "//a[contains(@href,'AcUgoqK4pdde') or contains(@href,'DGTH-DS-Panetones')]")
link.click()

# si abre en nueva pestaña/ventana, cambiar el foco a la última
driver.switch_to.window(driver.window_handles[-1])


In [7]:
import os
import time
import zipfile

# --- Parámetros ---
carpeta_descargas = os.path.expanduser("~/Downloads")
prefijo = "DGTH-DS-Panetones_Solidarios-_(E-commerce)"
carpeta_destino = r"C:\Trabajo"  # <- ruta final donde guardarás el CSV

# Crear la carpeta destino si no existe
os.makedirs(carpeta_destino, exist_ok=True)

# Esperar brevemente por si la descarga no terminó aún
time.sleep(3)

# 1️⃣ Buscar el ZIP que empiece con el prefijo
zips = [f for f in os.listdir(carpeta_descargas) if f.startswith(prefijo) and f.endswith(".zip")]
if not zips:
    raise FileNotFoundError("No se encontró ningún .zip con el prefijo indicado en Descargas.")

# Tomar el ZIP más reciente
zip_path = max((os.path.join(carpeta_descargas, f) for f in zips), key=os.path.getmtime)
print("📦 ZIP encontrado:", zip_path)

# 2️⃣ Abrir el ZIP y buscar el CSV
with zipfile.ZipFile(zip_path, 'r') as z:
    miembros = z.namelist()
    candidatos_csv = [m for m in miembros if os.path.basename(m).startswith(prefijo) and m.lower().endswith(".csv")]
    if not candidatos_csv:
        raise FileNotFoundError("No se encontró ningún archivo .csv dentro del ZIP con el prefijo indicado.")
    
    # Tomar el primero encontrado
    miembro_csv = candidatos_csv[0]
    ruta_salida = os.path.join(carpeta_destino, os.path.basename(miembro_csv))
    
    # 3️⃣ Extraer solo ese CSV
    with z.open(miembro_csv) as src, open(ruta_salida, "wb") as dst:
        dst.write(src.read())

print(f"✅ CSV extraído correctamente en: {ruta_salida}")


📦 ZIP encontrado: C:\Users\japhe/Downloads\DGTH-DS-Panetones_Solidarios-_(E-commerce)102825_01_26.zip
✅ CSV extraído correctamente en: C:\Trabajo\DGTH-DS-Panetones_Solidarios-_(E-commerce)102825_01_26_1.csv


In [13]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

# --- Ruta del archivo CSV ---
carpeta_destino = r"C:\Trabajo"
prefijo = "DGTH-DS-Panetones_Solidarios-_(E-commerce)"

# Buscar el CSV más reciente en esa carpeta
archivos_csv = [f for f in os.listdir(carpeta_destino) if f.startswith(prefijo) and f.endswith(".csv")]
if not archivos_csv:
    raise FileNotFoundError("No se encontró ningún CSV con el prefijo en C:\\Trabajo.")

csv_path = max((os.path.join(carpeta_destino, f) for f in archivos_csv), key=os.path.getmtime)
print("📂 Archivo CSV a subir:", csv_path)

# --- Abrir Kissflow en nueva pestaña ---
driver.execute_script("window.open('https://pucp.kissflow.com/view/dataset/BU_DGTH_DS_Panetones_Solidarios_E_commer', '_blank');")
driver.switch_to.window(driver.window_handles[-1])

time.sleep(5)  # esperar que cargue Kissflow

# 1️⃣ Click en “Importar CSV”
importar_btn = driver.find_element(By.XPATH, "//button[.//span[normalize-space()='Importar CSV']]")
importar_btn.click()
time.sleep(2)

# 2️⃣ Click en “Cargar”
cargar_btn = driver.find_element(By.XPATH, "//button[.//span[normalize-space()='Cargar']]")
cargar_btn.click()
time.sleep(2)

# 3️⃣ Buscar el input tipo “file” dentro de la opción “Su dispositivo”
input_file = driver.find_element(By.XPATH, "//li[contains(@class,'fileupload')]//input[@type='file']")

# 4️⃣ Enviar el archivo CSV (subida directa)
input_file.send_keys(csv_path)
print("✅ Archivo cargado en Kissflow:", csv_path)

time.sleep(5)  # esperar a que Kissflow procese la subida


📂 Archivo CSV a subir: C:\Trabajo\DGTH-DS-Panetones_Solidarios-_(E-commerce)102825_01_26_1.csv
✅ Archivo cargado en Kissflow: C:\Trabajo\DGTH-DS-Panetones_Solidarios-_(E-commerce)102825_01_26_1.csv
